In [96]:
import math
import numpy as np
from itertools import islice


## Atividade 01 - Capítulo 05 (Ruggiero)

Cálculos realizados para o exercício.

In [11]:

A = np.array([[1.00, 2.80, 3.84 ], 
              [1.00, 3.00, 9.00 ], 
              [1.00, 3.20, 10.24]], dtype=float)

b = np.array([16.44, 20.08, 24.53], dtype=float)

a = np.round(np.linalg.solve(A, b), 5)
a

array([-48.65367,  23.53112,  -0.20663])

## Atividade 09 - Capítulo 05 (Ruggiero)

#### Implementação algoritmica

In [106]:
class Interpolador():

    def __init__(self, decimals: int = 8, 
                 treshold_degree: float = 0.05
                 ):
        
        self.decimals = decimals
        self.treshold_degree = treshold_degree
        pass
    
    def _calcular_passo_inicial(self, 
                                X: np.array, 
                                y: np.array
                                ) -> list:
        """
        Estima passo inicial
        """
        _y = []
        n = X.shape[0]

        for i in range(n):
            if (i > 0):
                a = (y[i - 1] - y[i]) / (X[i - 1] - X[i])
                a = np.round(a, self.decimals)
                b = (i - 1, i)
                _y.append({a: b})
        
        return _y

    def _calcular_demais_passos(self, 
                                X: np.array, 
                                y: list
                                ) -> list:
        """
        Calcula demais passos
        """
        _y = []
        n = len(y)

        for i in range(n):
            if (i > 0):
                
                _xi_from = list(islice(y[i - 1].values(), 1))[0][0]
                _xi_to = list(islice(y[i].values(), 1))[0][1]

                _y_from = list(islice(y[i - 1].keys(), 1))[0]
                _y_to = list(islice(y[i].keys(), 1))[0]

                a = (_y_from - _y_to) / (X[_xi_from] - X[_xi_to])
                a = np.round(a, self.decimals)
                b = (_xi_from, _xi_to)

                _y.append({a: b})
        
        return _y
    
    def _get_tabela_diferencas(self, 
                               X: np.array, 
                               y: np.array
                               ) -> dict[int, list]:
        """
        Calcula tabela de diferenças
        """
        diferencas = {}
        
        diferencas[0] = [{np.float64(valor): (i, i)} for i, valor in enumerate(y)]

        _y = self._calcular_passo_inicial(X, y)
        diferencas[1] = _y

        grau = 2

        while True:
            try:
                _y = self._calcular_demais_passos(X, _y)
                if not _y: 
                    break
                diferencas[grau] = _y
            except:
                break

            grau += 1

        return diferencas
    
    def _escolher_polinomios(self,
                             tabela_diferencas: dict[int, list], 
                             ) -> int:
        """
        Escolhe polinômios baeado no treshold de validação.

        Para isto, utiliza o coeficiente de variação aplicado
        aos níveis da tabela de diferenças. 

        O primeiro polinõmio que seja menor que treshold, 
        é escolhido.

        """

        somas = []
        
        for diferencas in tabela_diferencas.values():
            soma = []
            for diferenca in diferencas:
                soma.append(list(islice(diferenca.keys(), 1))[0])
            soma = np.array(soma)
            soma = np.std(soma) / abs(np.mean(soma))
            somas.append(soma)

        for i, soma in enumerate(somas):
            if i > 0:
                if (i + 1) <= len(somas):
                    if (somas[i] <= self.treshold_degree):
                        print(f'Grau escolhido automaticamente: {i}')
                        return i
            
        print('Utilizando maior polinômio possível')
        return len(somas)

    def _estimar_M(self, 
                   tabela_diferencas: dict[int, list], 
                   degree: int
                   ) -> float:

        M = 0

        for i, diferencas in tabela_diferencas.items():
            if i <= (degree + 1):
                for diferenca in diferencas:
                    m = abs(list(islice(diferenca.keys(), 1))[0])
                    if m > M:
                        M = m

        return M

    def fit(self,
            X: np.array, 
            y: np.array,
            degree: int = None
            ) -> None:
        
        """
            Estima tabela de diferenças e realiza escolha de polinômio.
        """

        tabela_diferencas = self._get_tabela_diferencas(X, y)
        self.tabela_diferencas = tabela_diferencas
        
        self.degree = degree if degree else self._escolher_polinomios(tabela_diferencas)
        self.X = np.copy(X)
        self.y = np.copy(y)

        return
    
    def predict(self, 
                x: np.array
                ) -> np.array:
        
        if type(self.X) == type(None) \
        or type(self.X) == type(None) \
        or not self.degree:
            raise AttributeError('É necessário treinar o interpolador antes')

        distancias = np.abs(self.X - x)
        _y = self.y[np.argsort(distancias)[:self.degree + 1]]

        _X = self.X[np.argsort(distancias)[:self.degree + 1]]
        M = self._estimar_M(self.tabela_diferencas, self.degree)
        E = np.abs(np.prod(_X)) * (M / math.factorial(self.degree + 1))
        print(f'Erro estimado: |f(x) - p(x)| < {E}')

        k = 0

        n = _y.shape[0]
        A = np.zeros((n, self.degree + 1))

        A[:, 0] = 1
        for i in range(1, self.degree + 1):
            A[:, i] = _y ** (i + 1)

        a = np.linalg.solve(A, _y)
        self.a = a

        r = self.a[0]
        for i in range(1, self.degree + 1):
            r += self.a[i] * (x ** i)

        return r
    


#### Exemplo do capítulo 5.6.1

In [107]:

X = np.array([1, 1.01   , 1.02  , 1.030     , 1.04      , 1.05  ], dtype=float)
y = np.array([1, 1.005  , 1.01  , 1.0149    , 1.0198    , 1.0247], dtype=float)

interpolador = Interpolador(decimals=5, treshold_degree=0.05)
interpolador.fit(X, y)

interpolador.predict(1)


Grau escolhido automaticamente: 1
Erro estimado: |f(x) - p(x)| < 0.5174735


/tmp/ipykernel_62038/867832290.py:107: RuntimeWarning: divide by zero encountered in scalar divide
  soma = np.std(soma) / abs(np.mean(soma))


np.float64(1.0)

#### Exercício 09

**a. Estime o valor de f(1.23) da melhor maneira possível, de forma que se possa estimar o erro cometido.**


In [109]:

# Define vetores
X = np.array([0.0   , 0.5       , 1.0   , 1.5       , 2.0   , 2.5   ], dtype=np.float64)
y = np.array([-2.78 , -2.241    , -1.65 , -0.594    , 1.34  , 4.564 ], dtype=np.float64)

# Calcula interpolacao
interpolador = Interpolador(decimals=5, treshold_degree=0.05)
interpolador.fit(X, y)

# Imprime tabela de diferencas
print('\nTabela de diferenças: ')
for _ in interpolador.tabela_diferencas:
    print(_, interpolador.tabela_diferencas[_])

# Imprime a predicao
print()
print(f'p(x = 1.23) = {round(interpolador.predict(1.23), 5)}')



Grau escolhido automaticamente: 3

Tabela de diferenças: 
0 [{np.float64(-2.78): (0, 0)}, {np.float64(-2.241): (1, 1)}, {np.float64(-1.65): (2, 2)}, {np.float64(-0.594): (3, 3)}, {np.float64(1.34): (4, 4)}, {np.float64(4.564): (5, 5)}]
1 [{np.float64(1.078): (0, 1)}, {np.float64(1.182): (1, 2)}, {np.float64(2.112): (2, 3)}, {np.float64(3.868): (3, 4)}, {np.float64(6.448): (4, 5)}]
2 [{np.float64(0.104): (0, 2)}, {np.float64(0.93): (1, 3)}, {np.float64(1.756): (2, 4)}, {np.float64(2.58): (3, 5)}]
3 [{np.float64(0.55067): (0, 3)}, {np.float64(0.55067): (1, 4)}, {np.float64(0.54933): (2, 5)}]
4 [{np.float64(-0.0): (0, 4)}, {np.float64(-0.00067): (1, 5)}]
5 [{np.float64(-0.00027): (0, 5)}]

Erro estimado: |f(x) - p(x)| < 0.403
p(x = 1.23) = 0.62753


**b. Justifique o grau de polinômio que você escolheu para resolver o item a**

O cálculo foi implementado conforme o exemplo do capítulo 5.6.1. Nele, se utiliza o primeiro polinômio considerado constante, do menor para o maior.

Para isto, foi implementada uma lógica que varre toda a tabela de diferenças encontrando o coeficiente de variação (desvio padrão sobre a média), que seja menor do que um valor predefinido. Neste caso, 0.05. Assim, optou-se pelo polinômio 3.